In [27]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
pd.set_option('display.max_rows', 250)

In [3]:
df = pd.read_csv("newOutputs/classifiers.csv", index_col=0)

def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ["items","nodes","timesteps","arcs_ratio","values","PC1","PC2","PC3","PC4","PCweight1","PCweight2","PCweight3","PCweight4","PCAsAbove95","maxdegree/nodes","roots/nodes","leaves/nodes","longestpath/nodes","communities/nodes","mean_constraint","mean_efficiency","s_metric/arcs","mean_similarity","mean_nr_similarities/nodes","mean_degree","nodestates","arcs","items/nodestates","items/arcs"]
df[columns] = df[columns].apply(normalize)
df

,items,nodes,timesteps,arcs_ratio,values,algorithm,scorefunction,fscore,structure_id,PC1,...,mean_constraint,mean_efficiency,s_metric/arcs,mean_similarity,mean_nr_similarities/nodes,mean_degree,nodestates,arcs,items/nodestates,items/arcs
0,0.1,0.5,0.333333,0.499999,0.5,HC,BDEU,0.333333,0,0.079587,...,0.583333,0.750000,0.237154,0.25000,0.25000,0.45,0.000244,0.1875,0.100000,0.100000
1,0.1,0.5,0.333333,0.499999,0.5,HC,AIC,0.000000,0,0.066578,...,0.583333,0.750000,0.237154,0.25000,0.25000,0.45,0.000244,0.1875,0.100000,0.100000
2,0.1,0.5,0.333333,0.499999,0.5,HC,BIC,0.000000,0,0.079688,...,0.583333,0.750000,0.237154,0.25000,0.25000,0.45,0.000244,0.1875,0.100000,0.100000
3,0.1,0.5,0.333333,0.499999,0.5,HC,K2,0.000000,0,0.074820,...,0.583333,0.750000,0.237154,0.25000,0.25000,0.45,0.000244,0.1875,0.100000,0.100000
4,0.1,0.5,0.333333,0.499999,0.5,HC,L2L,0.000000,0,0.072970,...,0.583333,0.750000,0.237154,0.25000,0.25000,0.45,0.000244,0.1875,0.100000,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33295,1.0,1.0,0.333333,0.875000,1.0,MMHC,BDEU,0.000000,85,0.017435,...,0.462386,0.664444,0.843215,0.18941,0.34375,0.90,1.000000,0.8750,0.000244,0.214286
33296,1.0,1.0,0.333333,0.875000,1.0,MMHC,BIC,0.000000,85,0.019309,...,0.462386,0.664444,0.843215,0.18941,0.34375,0.90,1.000000,0.8750,0.000244,0.214286
33297,1.0,1.0,0.333333,0.875000,1.0,MMHC,K2,0.000000,85,0.016330,...,0.462386,0.664444,0.843215,0.18941,0.34375,0.90,1.000000,0.8750,0.000244,0.214286
33298,1.0,1.0,0.333333,0.875000,1.0,3OFF2,constraint,0.600000,85,0.016871,...,0.462386,0.664444,0.843215,0.18941,0.34375,0.90,1.000000,0.8750,0.000244,0.214286


In [4]:
dfhot = pd.get_dummies(df, columns=["algorithm","scorefunction"], dtype=int)
del df
dfhot

,items,nodes,timesteps,arcs_ratio,values,fscore,structure_id,PC1,PC2,PC3,...,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,scorefunction_AIC,scorefunction_BDEU,scorefunction_BIC,scorefunction_K2,scorefunction_L2L,scorefunction_constraint
0,0.1,0.5,0.333333,0.499999,0.5,0.333333,0,0.079587,0.015593,0.009940,...,1,0,0,0,0,1,0,0,0,0
1,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.066578,0.021655,0.004711,...,1,0,0,0,1,0,0,0,0,0
2,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.079688,0.017840,0.007377,...,1,0,0,0,0,0,1,0,0,0
3,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.074820,0.014163,0.005103,...,1,0,0,0,0,0,0,1,0,0
4,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.072970,0.020888,0.007101,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33295,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.017435,0.010863,0.007120,...,0,0,1,0,0,1,0,0,0,0
33296,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.019309,0.011259,0.007205,...,0,0,1,0,0,0,1,0,0,0
33297,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.016330,0.012039,0.006773,...,0,0,1,0,0,0,0,1,0,0
33298,1.0,1.0,0.333333,0.875000,1.0,0.600000,85,0.016871,0.013103,0.007061,...,0,0,0,0,0,0,0,0,0,1


In [36]:
# %%time
# def powerset(iterable):
#     "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
#     s = list(iterable)
#     combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
#     merged = []
#     for combination in combinations:
#         merged.append(np.concatenate(combination, axis=None))
#     return merged
    
# item_c = ["items"]
# nodes_c = ["nodes"]
# arc_c = ["arcs_ratio"]
# times_c = ["timesteps"]
# values_c = ["values"]
# algorithm_c = ["algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_MMHC","algorithm_TABU"]
# score_c = ["scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L"]
# nodestates = ["nodestates"]
# arcs = ["arcs"]
# items_nodestates = ["items/nodestates"]
# items_arcs = ["items/arcs"]
# PC1 = ["PC1"]
# PC2 = ["PC2"]
# PC3 = ["PC3"]
# PC4 = ["PC4"]
# PCweight1 = ['PCweight1']
# PCweight2 = ['PCweight2']
# PCweight3 = ['PCweight3']
# PCweight4 = ['PCweight4']
# PCAsAbove95 = ["PCAsAbove95"]
# maxdegree = ["maxdegree/nodes"]
# roots = ["roots/nodes"]
# leaves = ["leaves/nodes"] 
# longestpath = ["longestpath/nodes"]
# communities = ["communities/nodes"]

# columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs, PC1, PC2, PC3, PC4, PCweight1, PCweight2, PCweight3, PCweight4, PCAsAbove95, maxdegree, roots,leaves,longestpath,communities]
# column_sets = powerset(columns_c)

column_sets = [["items","nodes","timesteps","arcs_ratio","values","algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_TABU","scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L","PC1","PC2","PC3","PC4","PCweight1","PCweight2","PCweight3","PCweight4","PCAsAbove95","maxdegree/nodes","roots/nodes","leaves/nodes","longestpath/nodes","communities/nodes","mean_constraint","mean_efficiency","s_metric/arcs","mean_similarity","mean_nr_similarities/nodes","mean_degree","nodestates","arcs","items/nodestates","items/arcs"]]#"algorithm_MMHC",

results3 = pd.DataFrame()

for c, columns in enumerate(column_sets[::-1]):
    X_train, X_test, y_train, y_test = train_test_split(dfhot[columns], dfhot["fscore"], test_size = 0.2)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out(X_train.columns)), pd.DataFrame(poly.fit_transform(X_test), columns=poly.get_feature_names_out(X_test.columns))

    del X_train
    del X_test
    
    regr = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
#     regr = LinearRegression()
    regr.fit(X_train_poly, y_train)
    print(regr.feature_importances_)
    
    results3 =  pd.concat([results3, pd.DataFrame({"items":["items" in columns],"nodes":["nodes" in columns],"timesteps":["timesteps" in columns],"arcs_ratio":["arcs_ratio" in columns],"values":["values" in columns],"algorithm_3OFF2":["algorithm_3OFF2" in columns],"algorithm_HC":["algorithm_HC" in columns],"algorithm_MIIC":["algorithm_MIIC" in columns],"algorithm_MMHC":["algorithm_MMHC" in columns],"algorithm_TABU":["algorithm_TABU" in columns],"scorefunction_AIC":["scorefunction_AIC" in columns],"scorefunction_BDEU":["scorefunction_BDEU" in columns],"scorefunction_BIC":["scorefunction_BIC" in columns],"scorefunction_K2":["scorefunction_K2" in columns],"scorefunction_L2L":["scorefunction_L2L" in columns],"fscore":["fscore" in columns],"PC1":["algorithm_3OFF2" in columns],"PC2":["algorithm_3OFF2" in columns],"PC3":["algorithm_3OFF2" in columns],"PC4":["algorithm_3OFF2" in columns],"PCweight1":["algorithm_3OFF2" in columns],"PCweight2":["algorithm_3OFF2" in columns],"PCweight3":["algorithm_3OFF2" in columns],"PCweight4":["algorithm_3OFF2" in columns],"PCAsAbove95":["algorithm_3OFF2" in columns],"maxdegree/nodes":["algorithm_3OFF2" in columns],"roots/nodes":["algorithm_3OFF2" in columns],"leaves/nodes":["algorithm_3OFF2" in columns],"longestpath/nodes":["algorithm_3OFF2" in columns],"communities/nodes":["algorithm_3OFF2" in columns],"mean_constraint":["algorithm_3OFF2" in columns],"mean_efficiency":["algorithm_3OFF2" in columns],"s_metric/arcs":["algorithm_3OFF2" in columns],"mean_similarity":["algorithm_3OFF2" in columns],"mean_nr_similarities/nodes":["algorithm_3OFF2" in columns],"mean_degree":["algorithm_3OFF2" in columns],"nodestates":["algorithm_3OFF2" in columns],"arcs":["algorithm_3OFF2" in columns],"items/nodestates":["algorithm_3OFF2" in columns],"items/arcs":["algorithm_3OFF2" in columns], "score":[regr.score(X_test_poly, y_test)]})], ignore_index=True)
    print(c/len(column_sets))
    break

results3

[0.00000000e+00 3.08217022e-05 3.11137919e-05 2.38928405e-05
 3.51087765e-05 2.18589430e-05 4.00226709e-05 3.42577114e-05
 7.54682987e-05 4.98571644e-05 5.60287335e-05 7.82580028e-05
 5.96775826e-05 5.34891697e-05 5.91612989e-05 1.22362821e-04
 1.55285801e-04 1.78074464e-04 1.46407459e-04 2.58875283e-04
 4.08066495e-04 2.18124624e-04 2.06394863e-04 2.59673310e-04
 2.71917961e-05 6.39280697e-05 2.36316540e-04 4.09588138e-05
 0.00000000e+00 1.17180825e-04 1.83849901e-04 4.76745015e-04
 9.26935172e-05 1.29355874e-04 1.01183148e-04 9.42649131e-05
 7.89056940e-05 1.20163277e-04 1.78692950e-04 8.34134335e-05
 2.38752953e-04 9.92466696e-04 3.47127236e-04 9.51435140e-05
 3.10552306e-02 5.29439561e-02 1.84160694e-02 2.80201416e-02
 2.60635861e-04 4.46283317e-04 1.73756256e-04 2.27111654e-04
 3.27453949e-04 5.24995150e-04 4.69167950e-04 3.46234767e-04
 3.35020741e-04 5.06842567e-04 5.37861488e-04 3.89365188e-04
 4.05132567e-04 3.51722527e-04 5.52957761e-04 2.68214790e-04
 2.80469569e-04 3.207590

,items,nodes,timesteps,arcs_ratio,values,algorithm_3OFF2,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,...,mean_efficiency,s_metric/arcs,mean_similarity,mean_nr_similarities/nodes,mean_degree,nodestates,arcs,items/nodestates,items/arcs,score
0,True,True,True,True,True,True,True,True,False,True,...,True,True,True,True,True,True,True,True,True,0.577315


In [37]:
cm = sns.light_palette("red", as_cmap=True)

results3["items"] = results3["items"].astype(int)
results3["nodes"] = results3["nodes"].astype(int)
results3["arcs_ratio"] = results3["arcs_ratio"].astype(int)
results3["timesteps"] = results3["timesteps"].astype(int)
results3["values"] = results3["values"].astype(int)
# results3["algorithm"] = results3["algorithm"].astype(int)
# results3["scorefunction"] = results3["scorefunction"].astype(int)
results3["nodestates"] = results3["nodestates"].astype(int)
results3["arcs"] = results3["arcs"].astype(int)
results3["items/nodestates"] = results3["items/nodestates"].astype(int)
results3["items/arcs"] = results3["items/arcs"].astype(int)
results3["PC1"] = results3["PC1"].astype(int)
results3["PC2"] = results3["PC2"].astype(int)
results3["PC3"] = results3["PC3"].astype(int)
results3["PC4"] = results3["PC4"].astype(int)
results3['PCweight1'] = results3['PCweight1'].astype(int)
results3['PCweight2'] = results3['PCweight2'].astype(int)
results3['PCweight3'] = results3['PCweight3'].astype(int)
results3['PCweight4'] = results3['PCweight4'].astype(int)
results3["PCAsAbove95"] = results3["PCAsAbove95"].astype(int)
results3["maxdegree/nodes"] = results3["maxdegree/nodes"].astype(int)
results3["roots/nodes"] = results3["roots/nodes"].astype(int)
results3["leaves/nodes"] = results3["leaves/nodes"].astype(int) 
results3["longestpath/nodes"] = results3["longestpath/nodes"].astype(int)
results3["communities/nodes"] = results3["communities/nodes"].astype(int)

# results3.sort_values("scorefunction", ascending=False).to_csv("hotlinreg.csv")
display(results3[['items', 'nodes', 'timesteps', 'arcs_ratio', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs','PC1','PC2','PC3','PC4','PCweight1','PCweight2','PCweight3','PCweight4','PCAsAbove95', "maxdegree/nodes", "roots/nodes", "leaves/nodes", "longestpath/nodes","communities/nodes","score"]].sort_values("score", ascending=False).head().style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1))
# display(results3[['items', 'nodes', 'timesteps', 'arcs_ratio', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs','PC1','PC2','PC3','PC4','PCweight1','PCweight2','PCweight3','PCweight4','PCAsAbove95', "maxdegree/nodes", "roots/nodes", "leaves/nodes", "longestpath/nodes","communities/nodes","score"]].sort_values("score", ascending=False).tail().style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1))


,items,nodes,timesteps,arcs_ratio,values,nodestates,arcs,items/nodestates,items/arcs,PC1,PC2,PC3,PC4,PCweight1,PCweight2,PCweight3,PCweight4,PCAsAbove95,maxdegree/nodes,roots/nodes,leaves/nodes,longestpath/nodes,communities/nodes,score
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0.577315


 # Feature Importances

In [39]:
#"algorithm_MMHC",
columns = ["items","nodes","timesteps","arcs_ratio","values","algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_TABU","scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L","PC1","PC2","PC3","PC4","PCweight1","PCweight2","PCweight3","PCweight4","PCAsAbove95","maxdegree/nodes","roots/nodes","leaves/nodes","longestpath/nodes","communities/nodes","mean_constraint","mean_efficiency","s_metric/arcs","mean_similarity","mean_nr_similarities/nodes","mean_degree","nodestates","arcs","items/nodestates","items/arcs"]

feature_importance = pd.DataFrame()

for rep in range(5):
    X_train, X_test, y_train, y_test = train_test_split(dfhot[columns], dfhot["fscore"], test_size = 0.2)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out(X_train.columns)), pd.DataFrame(poly.fit_transform(X_test), columns=poly.get_feature_names_out(X_test.columns))
    
    regr = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
    #     regr = LinearRegression()
    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

    # np.max(np.abs(regr.coef_))
    if len(feature_importance) < 1:
        feature_importance = pd.concat([feature_importance,pd.DataFrame({'Feature': X_train_poly.columns, 'Importance': regr.feature_importances_})],axis=1)
    else:
        feature_importance = pd.merge(feature_importance,pd.DataFrame({'Feature': X_train_poly.columns, 'Importance': regr.feature_importances_}), on="Feature")

feature_importance

0.5802158352047411
0.5719433270888638
0.5850543734642393
0.5671889435786315


MergeError: Passing 'suffixes' which cause duplicate columns {'Importance_x'} is not allowed.